# End-to-End Runbook

This notebook runs the full GridPulse workflow: data → features → splits → training → API smoke test.

In [1]:
from pathlib import Path
import sys
repo_root = Path.cwd().parent if Path.cwd().name == 'notebooks' else Path.cwd()
print('Python:', sys.executable)
print('repo_root:', repo_root)

Python: /Users/pratik_n/Downloads/gridpulse/.venv/bin/python
repo_root: /Users/pratik_n/Downloads/gridpulse


## 1) Install local package (editable)

In [2]:
import subprocess
venv_python = repo_root / '.venv' / 'bin' / 'python'
python_bin = str(venv_python) if venv_python.exists() else 'python'
subprocess.run([python_bin, '-m', 'pip', 'install', '-e', str(repo_root)], check=False)

Obtaining file:///Users/pratik_n/Downloads/gridpulse
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Checking if build backend supports build_editable: started
  Checking if build backend supports build_editable: finished with status 'done'
  Getting requirements to build editable: started
  Getting requirements to build editable: finished with status 'done'
  Preparing editable metadata (pyproject.toml): started
  Preparing editable metadata (pyproject.toml): finished with status 'done'
  Building editable for gridpulse (pyproject.toml): started
  Building editable for gridpulse (pyproject.toml): finished with status 'done'
  Created wheel for gridpulse: filename=gridpulse-0.1.0-0.editable-py3-none-any.whl size=4315 sha256=4b52389ba5424853363740744583890b0dbbdffc1a375b11d5d429432826cba1
  Stored in directory: /private/var/folders/pk/ksqc41pj60q3589gfwhqz5tw0000gn/T/pip-ephem-wheel-cache-g8fp_s9x/wheels/8a/44/a7/bd98a379fbc961575d

CompletedProcess(args=['/Users/pratik_n/Downloads/gridpulse/.venv/bin/python', '-m', 'pip', 'install', '-e', '/Users/pratik_n/Downloads/gridpulse'], returncode=0)

## 2) Ensure raw OPSD file exists

In [3]:
from pathlib import Path
raw_dir = repo_root / 'data' / 'raw'
raw_csv = raw_dir / 'time_series_60min_singleindex.csv'
nested = raw_dir / 'opsd-time_series-2020-10-06' / 'time_series_60min_singleindex.csv'
if raw_csv.exists():
    print('Raw file present:', raw_csv)
elif nested.exists():
    raw_dir.mkdir(parents=True, exist_ok=True)
    raw_csv.write_bytes(nested.read_bytes())
    print('Copied raw file from nested OPSD folder')
else:
    print('Raw OPSD file not found. Download or place it at:')
    print(raw_csv)

Raw file present: /Users/pratik_n/Downloads/gridpulse/data/raw/time_series_60min_singleindex.csv


## 3) Run data pipeline

In [4]:
import subprocess
subprocess.run(['python', '-m', 'gridpulse.data_pipeline.validate_schema', '--in', str(repo_root/'data'/'raw'), '--report', str(repo_root/'reports'/'data_quality_report.md')], check=False)
subprocess.run(['python', '-m', 'gridpulse.data_pipeline.build_features', '--in', str(repo_root/'data'/'raw'), '--out', str(repo_root/'data'/'processed')], check=False)
subprocess.run(['python', '-m', 'gridpulse.data_pipeline.split_time_series', '--in', str(repo_root/'data'/'processed'/'features.parquet'), '--out', str(repo_root/'data'/'processed'/'splits')], check=False)

Wrote report: /Users/pratik_n/Downloads/gridpulse/reports/data_quality_report.md
Saved: /Users/pratik_n/Downloads/gridpulse/data/processed/features.parquet
Rows: 49481 | Columns: 39
Wrote splits to: /Users/pratik_n/Downloads/gridpulse/data/processed/splits


CompletedProcess(args=['python', '-m', 'gridpulse.data_pipeline.split_time_series', '--in', '/Users/pratik_n/Downloads/gridpulse/data/processed/features.parquet', '--out', '/Users/pratik_n/Downloads/gridpulse/data/processed/splits'], returncode=0)

## 4) Train forecasting models

In [5]:
from pathlib import Path
features_path = repo_root / 'data' / 'processed' / 'features.parquet'
if not features_path.exists():
    print('features.parquet not found. Run step 3 first.')
else:
    import subprocess
    subprocess.run(['python', '-m', 'gridpulse.forecasting.train', '--config', str(repo_root/'configs'/'train_forecast.yaml')], check=False)

Traceback (most recent call last):
  File "/Applications/Xcode.app/Contents/Developer/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/runpy.py", line 197, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/Applications/Xcode.app/Contents/Developer/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "/Users/pratik_n/Downloads/gridpulse/src/gridpulse/forecasting/train.py", line 365, in <module>
    main()
  File "/Users/pratik_n/Downloads/gridpulse/src/gridpulse/forecasting/train.py", line 177, in main
    raise FileNotFoundError(f"Missing {features_path}. Run build_features first.")
FileNotFoundError: Missing data/processed/features.parquet. Run build_features first.


## 5) Start API (separate terminal)

```bash
uvicorn services.api.main:app --reload --port 8000
```

## 6) API smoke test

In [6]:
import requests
base = 'http://localhost:8000'
try:
    print(requests.get(f'{base}/health', timeout=5).json())
except Exception as e:
    print('API not reachable. Start it in a separate terminal.', e)

API not reachable. Start it in a separate terminal. HTTPConnectionPool(host='localhost', port=8000): Max retries exceeded with url: /health (Caused by NewConnectionError("HTTPConnection(host='localhost', port=8000): Failed to establish a new connection: [Errno 61] Connection refused"))


/Users/pratik_n/Downloads/gridpulse/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [7]:
import requests
base = 'http://localhost:8000'
try:
    print(requests.get(f'{base}/forecast', timeout=5).json().get('meta', {}))
except Exception as e:
    print('Forecast call failed:', e)

Forecast call failed: HTTPConnectionPool(host='localhost', port=8000): Max retries exceeded with url: /forecast (Caused by NewConnectionError("HTTPConnection(host='localhost', port=8000): Failed to establish a new connection: [Errno 61] Connection refused"))


In [8]:
import requests
base = 'http://localhost:8000'
payload = {
    'forecast_load_mw': [8000, 8200, 8100],
    'forecast_renewables_mw': [3200, 3100, 3300]
}
try:
    print(requests.post(f'{base}/optimize', json=payload, timeout=10).json())
except Exception as e:
    print('Optimize call failed:', e)

Optimize call failed: HTTPConnectionPool(host='localhost', port=8000): Max retries exceeded with url: /optimize (Caused by NewConnectionError("HTTPConnection(host='localhost', port=8000): Failed to establish a new connection: [Errno 61] Connection refused"))


## 7) Start the dashboard

```bash
cd frontend && npm run dev
```